# Trabajo Final - Criptografía Práctica y Blockchain

|    Nombre   |  Apellido  |     Padrón      |
|:-----------:|:----------:|:------------:|
|    Tomás     |   Rodriguez Dala    |    |
|    Mateo    |   Capón Blanquer    |  104258  |

## Enunciado
El objetivo de esta práctica es poner en conjunto varios de los temas vistos en la asignatura, a partir del uso de una biblioteca para pruebas de conocimiento nulo. En este caso, se utilizarán las bibliotecas https://github.com/lambdaclass/lambdaworks_cairo_prover/ y https://github.com/lambdaclass/lambdaworks/

Se proponen las siguientes consignas:
1. Crear el AIR para un cálculo simple (aplicación repetitiva de una función polinómica) y,
utilizando el prover, generar una prueba de validez computacional para el problema
seleccionado.
2. Desarrollar un programa que permita un cálculo utilizando el lenguaje CAIRO (sin utilizar
built-ins), generar una prueba de validez computacional y verificarla mediante el
programa verificador.
3. Explicar el diagrama de flujo para generar y verificar pruebas utilizando STARKs.
4. Analizar el efecto que tiene la cantidad de chequeos (queries) y blowup factor sobre el
tiempo de generación de la prueba y la memoria que ocupa la prueba.
5. Explicar los parámetros de los que depende la seguridad de la prueba ¿Cuáles son las
consecuencias de utilizar cuerpos más pequeños que el original?
6. ¿De qué manera se podría disminuir la memoria requerida por la prueba?


## Imports

In [2]:
:dep lambdaworks-stark = { path = "." }
:dep lambdaworks-math = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep lambdaworks-crypto = { git = "https://github.com/lambdaclass/lambdaworks", rev = "a17b951" }
:dep num-integer

In [3]:
use std::ops::Range;

use lambdaworks_math::field::fields::{
    fft_friendly::stark_252_prime_field::Stark252PrimeField as F,
    u64_prime_field::{F17, FE17},
};
use lambdaworks_stark::{
    cairo::{
        air::{
            generate_cairo_proof, verify_cairo_proof, MemorySegment, MemorySegmentMap,
            PublicInputs, FRAME_DST_ADDR, FRAME_OP0_ADDR, FRAME_OP1_ADDR, FRAME_PC,
        },
        cairo_layout::CairoLayout,
        execution_trace::build_main_trace,
        runner::run::{
            cairo0_program_path, cairo1_program_path, generate_prover_args, run_program,
            CairoVersion,
        },
    },
    starks::{
        example::{
            dummy_air::{self, DummyAIR},
            fibonacci_2_columns::{self, Fibonacci2ColsAIR},
            fibonacci_rap::{fibonacci_rap_trace, FibonacciRAP, FibonacciRAPPublicInputs},
            quadratic_air::{self, QuadraticAIR, QuadraticPublicInputs},
            simple_fibonacci::{self, FibonacciAIR, FibonacciPublicInputs},
        },
        proof::options::{ProofOptions, SecurityLevel},
        prover::prove,
        trace::TraceTable,
        verifier::verify,
    },
    FE,
};

use lambdaworks_crypto::fiat_shamir::transcript::Transcript;
use lambdaworks_math::field::{element::FieldElement, traits::IsFFTField};

use lambdaworks_stark::starks::{
    constraints::boundary::{BoundaryConstraint, BoundaryConstraints},
    context::AirContext,
    frame::Frame,
    proof::options::ProofOptions,
    trace::TraceTable,
    traits::AIR,
};
use std::sync::atomic::{AtomicUsize, AtomicU64, AtomicIsize, Ordering::SeqCst};
use std::time::{Instant, Duration};
use num_integer::binomial;

## Ejercicio 1

Crear el AIR para un cálculo simple (aplicación repetitiva de una función polinómica) y,
utilizando el prover, generar una prueba de validez computacional para el problema
seleccionado.

### Resolución

Utilizamos la siguiente relación de recurrencia:

$$
P_n = 2 \cdot P_{n-1} + P_{n - 2},
$$
junto con las siguientes condiciones de contorno:

$$
P_0 = 0,
P_1 = 1
$$

Explicamos lo que debimos implementar del AIR.

En principio para hacer un cálculo simple, de una sola columna, se eligen los siguientes valores en el contexto:

1. `transition_degrees = vec![1]`, porque la restricción es de grado 1.
2. `transition_offsets = vec![0, 1, 2]`, porque se necesitan las dos anteriores filas para calcular el valor actual.
3. `num_transition_constraints = 1`, porque hay solo una restricción de transición que define a la relación.
4. `transition_exemptions = vec![2]`, porque se deben exceptuar dos restricciones de transicion (por como está definida la restricción de transición).


Luego, para computar la transición, simplemente despejamos la relación de recurrencia a un lado del igual. Y por último, las restricciones de contorno son las indicadas anteriormente. Suponiendo que lo hacemos con una sola columna, las restricciones son simples de definir.

A continuación mostramos una implementación del AIR, genérico para aceptar n columnas y distintos valores de los coeficientes de la restricción de transición. Como no quisimos modificar el código del trait AIR, decidimos usar variables globales como solución rápida y fácil para la configuración del trait. De este modo, podemos modificar las variables globales N_COLS, FIRST_FACTOR y SECOND_FACTOR, para poder usar al AIR de distintos modos.


#### Variables para hacer el AIR un poco más genérico

In [4]:
// Cantidad de columnas que se van a usar en el AIR.
static N_COLS: AtomicUsize = AtomicUsize::new(1);

// Factores en la restriccion de transicion. P_n = FF * P_{n -1} + SF * P_{n -1}
static FIRST_FACTOR: AtomicU64 = AtomicU64::new(2);
static SECOND_FACTOR: AtomicIsize = AtomicIsize::new(1);

pub fn get_factors() -> (u64, isize) {
    (FIRST_FACTOR.load(SeqCst), SECOND_FACTOR.load(SeqCst))
}

pub fn change_n_cols(new_value: usize) -> usize {
    N_COLS.store(new_value, SeqCst);
    new_value
}

pub fn get_n_cols() -> usize {
    N_COLS.load(SeqCst)
}

// Esta funcion la utilizamos en pruebas nuestras para ver como se comportan los algoritmos
// si se elige una subsecuencia de la relacion. 
// Por ejemplo, en vez de tener la subsecuencia 1,2,3,4,5; con un jumpFactor de 2,
// se tendrian solo los numeros 1,3,5,...
// Aplicamos una idea similar pero para la relacion P_n = 2 P_{n-1} + P_{n-2}
pub fn update_factors(jump_factor: u32) -> (FieldElement<F>, FieldElement<F>) {
    let mut first_factor = 0;
    // Las raices del polinomio caracteristo son 1 + srqrt(2) y 1 - srqrt(2).
    // Por lo tanto, la relacion se puede expresar como 
    // P_n = a (1 + srqrt(2))^n + b (1 - srqrt(2))^n.
    // Si buscamos una subrelacion -una relacion que saltee algunos elementos de
    // la relacion para llegar a un resultado en menos pasos-, podemos pensar en un 
    // P_n*c = a (1 + srqrt(2))^nc + b (1 - srqrt(2))^nc. (c es un factor de escala o jump_factor)
    // Esta ecuacion tiene un polinomio caracteristico con raices en 
    // (1 + srqrt(2))^c y (1 - srqrt(2))^c, y su polinomio caracteristico se puede escribir como
    // x^2 - [(1 + srqrt(2))^c + (1 - srqrt(2))^c] x + (-1)^c.
    // De esto se obtienen el first_factor y second_factor. Para no tener que hacer (1 - srqrt(2))^c,
    // desarrollamos el coeficiente para hacer el calculo con enteros siempre (lo que se hace a continuacion).
    for k in 0..(jump_factor + 1) {
        if k % 2 == 0 {
            first_factor += binomial(jump_factor, k) * u32::pow(2, k / 2);
        }
    }
    first_factor *= 2;
    let second_factor = if jump_factor % 2 == 1 {1} else {-1};
    FIRST_FACTOR.store(first_factor as u64, SeqCst);
    SECOND_FACTOR.store(second_factor, SeqCst);
    let c0 = FieldElement::<F>::from(first_factor as u64);
    let c1 = if second_factor == -1 {
        FieldElement::<F>::zero() - FieldElement::<F>::one()
    } else {// Vale 1.
        FieldElement::<F>::one()
    };
    (c0, c1)
}

#### Implementación del AIR

In [5]:
#[derive(Clone)]
pub struct PellAIR<F>
where
    F: IsFFTField,
{
    context: AirContext,
    trace_length: usize,
    pub_inputs: PellPublicInputs<F>,
    n_columns: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
}

#[derive(Clone, Debug)]
pub struct PellPublicInputs<F>
where
    F: IsFFTField,
{
    pub a0: FieldElement<F>,
    pub a1: FieldElement<F>,
}

impl<F> AIR for PellAIR<F>
where
    F: IsFFTField,
{
    type Field = F;
    type RAPChallenges = ();
    type PublicInputs = PellPublicInputs<Self::Field>;

    fn new(
        trace_length: usize,
        pub_inputs: &Self::PublicInputs,
        proof_options: &ProofOptions,
    ) -> Self {
        let n_columns = get_n_cols();
        // Solo para el caso de una columna se necesitan las dos filas anteriores.
        // Para mas de una columna, se necesitan los valores en la ultima fila y en 
        // la actual.
        let transition_offsets = if n_columns > 1 {vec![0, 1]} else {vec![0, 1, 2]};
        
        // en total siempre se deben exceptuar dos restricciones de transicion. En el caso de 
        // una sola columna, son los dos primeros valores por ser condiciones de contorno. Si 
        // se tiene mas de una columna, estas restricciones estaran en la primera y la segunda
        // columna. Los demas, sí deben ser evaluados.
        let transition_exemptions = if n_columns == 1 {
            vec![2] 
        } else {
            vec![1, 1].into_iter().chain(std::iter::repeat(0).take(n_columns - 2)).collect()
        }; 
        
        let context = AirContext {
            proof_options: proof_options.clone(),
            trace_columns: n_columns,
            transition_degrees: vec![1; n_columns], // usamos la misma transicion en cada columna. 
            transition_offsets: transition_offsets,
            num_transition_constraints: n_columns,
            transition_exemptions: transition_exemptions,
            num_transition_exemptions: 1,
        };
        // Traemos las variables globales aca. 
        // Sabemos que no es muy bonito a nivel programacion, pero queriamos analizar distintas metricas
        // para distintos factores en las transiciones y nos parecio lo mas sencillo.
        let (c0, c1) = get_factors(); 
        let second_factor = if c1 < 0 {
            FieldElement::<F>::zero() - FieldElement::<F>::from((c1 * (-1)) as u64)
        } else {// Es usize porque c1 > 0.
            FieldElement::<F>::from(c1 as u64)
        };
        Self {
            pub_inputs: pub_inputs.clone(),
            context,
            trace_length,
            n_columns,
            first_factor: FieldElement::<F>::from(c0),
            second_factor,
        }
    }

    fn composition_poly_degree_bound(&self) -> usize {
        self.trace_length()
    }

    fn build_auxiliary_trace(
        &self,
        _main_trace: &TraceTable<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> TraceTable<Self::Field> {
        TraceTable::empty()
    }

    fn build_rap_challenges<T: Transcript>(&self, _transcript: &mut T) -> Self::RAPChallenges {}

    fn compute_transition(
        &self,
        frame: &Frame<Self::Field>,
        _rap_challenges: &Self::RAPChallenges,
    ) -> Vec<FieldElement<Self::Field>> {
        let first_row = frame.get_row(0);
        let second_row = frame.get_row(1);
        if self.n_columns > 1 {
            let mut transitions = vec![];
            // La primera restriccion es tal que s_{0, i+1} = FF * s_{ncols - 1, i} + SF * s_{ncols - 2, i}.
            transitions.push(second_row[0].clone() - 
                 self.first_factor.clone() * first_row[self.n_columns - 1].clone() - 
                 self.second_factor.clone() * first_row[(2 * self.n_columns - 2) % self.n_columns].clone());
            
            // La segunda restriccion tambien usa la fila anterior siempre. 
            transitions.push(second_row[1].clone() - 
                self.first_factor.clone() * second_row[0].clone() - 
                self.second_factor.clone() * first_row[self.n_columns - 1].clone());
            
            // Las demas transiciones son sobre la misma fila pero de las anteriores dos columnas.
            for i in 2..self.n_columns {
                transitions.push(second_row[i].clone() - 
                    self.first_factor.clone() * second_row[i-1].clone() - 
                    self.second_factor.clone() * second_row[i-2].clone());
            }
            return transitions;
        }
        // Cuando hay una sola columna necesitamos la fila anterior para poder calcular la restriccion.
        let third_row = frame.get_row(2);

        vec![third_row[0].clone() - 
            self.first_factor.clone() * second_row[0].clone() - 
            self.second_factor.clone() * first_row[0].clone()]
    }

    fn boundary_constraints(
        &self,
        _rap_challenges: &Self::RAPChallenges,
    ) -> BoundaryConstraints<Self::Field> {
        let a0 = BoundaryConstraint::new(0, 0, self.pub_inputs.a0.clone());
        // si se hace con mas de una columna, la segunda restriccion es el primer
        // valor de la segunda columna.
        let a1 = if self.n_columns > 1 { 
            BoundaryConstraint::new(1, 0, self.pub_inputs.a1.clone())
        } else { 
            BoundaryConstraint::new(0, 1, self.pub_inputs.a1.clone())
        };

        BoundaryConstraints::from_constraints(vec![a0, a1])
    }

    fn number_auxiliary_rap_columns(&self) -> usize {
        0
    }

    fn context(&self) -> &AirContext {
        &self.context
    }

    fn trace_length(&self) -> usize {
        self.trace_length
    }

    fn pub_inputs(&self) -> &Self::PublicInputs {
        &self.pub_inputs
    }
}

#### Cálculo de la Traza

In [6]:
pub fn pell_trace<F: IsFFTField>(
    initial_values: [FieldElement<F>; 2],
    trace_length: usize,
    n_columns: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
) -> TraceTable<F> {
    let mut ret: Vec<Vec<FieldElement<F>>> = vec![vec![]; n_columns];
    ret[0].push(initial_values[0].clone());
    
    // si hay mas de una columna, es el primer elemento de la segunda columna.
    // Sino, en la unica columna que hay 
    ret[1 % n_columns].push(initial_values[1].clone());
    let mut a_n_1 = initial_values[1].clone();
    let mut a_n_2 = initial_values[0].clone();

    for i in 2..(trace_length * n_columns) {
        let res = first_factor.clone() * a_n_1.clone() + second_factor.clone() * a_n_2.clone();
        ret[i % n_columns].push(res.clone());
        a_n_2 = a_n_1;
        a_n_1 = res;
    }
    TraceTable::new_from_cols(&ret)
}

#### Prueba de Validez

In [7]:
fn run_test_prove(
    n_cols: usize,
    trace_length: usize,
    first_factor: FieldElement<F>,
    second_factor: FieldElement<F>,
    pub_inputs: PellPublicInputs<F>,
    // este parametro lo usamos a modo de validacion, pero no es necesario en la prueba de validez.
    expected_result: FieldElement<F>, 
) -> (Duration, Duration, Duration) {
    let mut now = Instant::now();
    let trace = pell_trace([pub_inputs.a0.clone(), pub_inputs.a1.clone()], 
                            trace_length, n_cols, first_factor, second_factor);

    let trace_time = now.elapsed();
    
    // esta validacion la hacemos nosotros a modo de prueba para cuando 
    // usamos mas columnas y distintos factores.
    assert_eq!(expected_result, trace.get(trace_length - 1, n_cols - 1));
    
    let proof_options = ProofOptions::default_test_options();
    
    now = Instant::now();
    let proof = prove::<F, PellAIR<F>>(&trace, &pub_inputs, &proof_options).unwrap();
    let proof_time = now.elapsed();
    
    now = Instant::now();
    assert!(verify::<F, PellAIR<F>>(
        &proof,
        &pub_inputs,
        &proof_options
    ));
    let verify_time = now.elapsed();
    
    (trace_time, proof_time, verify_time)
}

In [12]:
let original_pub_inputs = PellPublicInputs {
    a0: FE::zero(),
    a1: FE::one(),
};
let n_cols = 1;
// calculamamos el P_1023.
let trace_length = 1024; 

// es nuestro valor de jumps para buscar subsecuencias. 
// Al ponerlo en 1, usamos la secuencia tal como la definimos arriba: P_n = 2 P_{n-1} + P_{n-2}.
let n_jumps = 1; 
let (original_c0, original_c1) = update_factors(n_jumps);

// Esta validacion la hacemos nosotros a modo de prueba para cuando usamos mas columnas y distintos factores.
// No es parte del proceso de la prueba de validez.
// Es para validar que todas las opciones que se nos ocurren
// para hacer el mismo calculo obtienen el mismo resultado.
let trace = pell_trace([original_pub_inputs.a0.clone(), 
                        original_pub_inputs.a1.clone()],
                        trace_length, n_cols, original_c0.clone(), original_c1.clone());
let expected_result = trace.get(trace_length - 1, n_cols - 1);

run_test_prove(n_cols, trace_length, original_c0.clone(), original_c1.clone(),
               original_pub_inputs.clone(), expected_result.clone())

(81.898µs, 40.060806ms, 2.610933ms)

## Ejercicio 2
Desarrollar un programa que permita un cálculo utilizando el lenguaje CAIRO (sin utilizar built-ins), generar una prueba de validez computacional y verificarla mediante el programa verificador.

### Resolución


## Ejercicio 3
Explicar el diagrama de flujo para generar y verificar pruebas utilizando STARKs.

### Resolución


## Ejercicio 4
Analizar el efecto que tiene la cantidad de chequeos (queries) y blowup factor sobre el tiempo de generación de la prueba y la memoria que ocupa la prueba.
### Resolución


## Ejercicio 5
Explicar los parámetros de los que depende la seguridad de la prueba ¿Cuáles son las consecuencias de utilizar cuerpos más pequeños que el original?


## Ejercicio 6
¿De qué manera se podría disminuir la memoria requerida por la prueba?

### Resolución

## ANEXO

In [13]:
let trace_length = 1024;
let power_2_tracelen = 10;

fn update_public_inputs(
    jump_len: u32,
    original_pub_inputs: PellPublicInputs<F>,
    trace: TraceTable<F>
) -> PellPublicInputs<F> {
    if jump_len == 1 {
        return original_pub_inputs;
    }
    println!("{}", trace.get(jump_len as usize, 0));
    PellPublicInputs {
        a0: trace.get((jump_len - 1) as usize, 0),
        a1: trace.get((2 * jump_len - 1) as usize, 0),
    }
}

let n_runs = 10;
for jump_factor in 0..power_2_tracelen {
    let jump_len = u32::pow(2,jump_factor);
    // cambia la ecuacion de recurrencia al ser una subsecuencia.
    let (c0, c1) = update_factors(jump_len);
    let pub_inputs = update_public_inputs(jump_len, original_pub_inputs.clone(), trace.clone());
    
    for col_factor in 0..power_2_tracelen-jump_factor+1 {
        let n_cols = change_n_cols(usize::pow(2,col_factor));
        let trace_length = usize::pow(2,power_2_tracelen - col_factor - jump_factor);
        let (mut trace_time, mut proof_time, mut verify_time) = 
                            (Duration::ZERO, Duration::ZERO, Duration::ZERO);  
        println!("#JUMPS: {} - # COLS: {} - #TRACELEN: {} - INPUT: {}", 
                  jump_len, n_cols, trace_length, pub_inputs.a1.clone());
        for _i in 0..n_runs {
            let (t, p, v) = run_test_prove(n_cols, trace_length, c0.clone(),
                                           c1.clone(), pub_inputs.clone(),
                                           expected_result.clone());
            trace_time += t; proof_time += p; verify_time += v;
        }
        println!("#JUMPS: {} - # COLS: {} #TRACELEN: {} - Tiempos: {}, {}, {}",
                  jump_len, n_cols, trace_length, trace_time.as_micros(), 
                  proof_time.as_millis(), verify_time.as_millis());

    }

}

#JUMPS: 1 - # COLS: 1 - #TRACELEN: 1024 - INPUT: 0x1
#JUMPS: 1 - # COLS: 1 #TRACELEN: 1024 - Tiempos: 743, 385, 26
#JUMPS: 1 - # COLS: 2 - #TRACELEN: 512 - INPUT: 0x1
#JUMPS: 1 - # COLS: 2 #TRACELEN: 512 - Tiempos: 704, 199, 14
#JUMPS: 1 - # COLS: 4 - #TRACELEN: 256 - INPUT: 0x1
#JUMPS: 1 - # COLS: 4 #TRACELEN: 256 - Tiempos: 751, 110, 8
#JUMPS: 1 - # COLS: 8 - #TRACELEN: 128 - INPUT: 0x1
#JUMPS: 1 - # COLS: 8 #TRACELEN: 128 - Tiempos: 709, 70, 5
#JUMPS: 1 - # COLS: 16 - #TRACELEN: 64 - INPUT: 0x1
#JUMPS: 1 - # COLS: 16 #TRACELEN: 64 - Tiempos: 778, 51, 4
#JUMPS: 1 - # COLS: 32 - #TRACELEN: 32 - INPUT: 0x1
#JUMPS: 1 - # COLS: 32 #TRACELEN: 32 - Tiempos: 801, 43, 4
#JUMPS: 1 - # COLS: 64 - #TRACELEN: 16 - INPUT: 0x1
#JUMPS: 1 - # COLS: 64 #TRACELEN: 16 - Tiempos: 871, 40, 5
#JUMPS: 1 - # COLS: 128 - #TRACELEN: 8 - INPUT: 0x1
#JUMPS: 1 - # COLS: 128 #TRACELEN: 8 - Tiempos: 897, 44, 7
#JUMPS: 1 - # COLS: 256 - #TRACELEN: 4 - INPUT: 0x1
#JUMPS: 1 - # COLS: 256 #TRACELEN: 4 - Tiempos: 773, 

thread '<unnamed>' panicked at 'attempt to multiply with overflow', src/lib.rs:213:29
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8ede3aae28fe6e4d52b38157d7bfe0d3bceef225/library/std/src/panicking.rs:593:5
   1: core::panicking::panic_fmt
             at /rustc/8ede3aae28fe6e4d52b38157d7bfe0d3bceef225/library/core/src/panicking.rs:67:14
   2: core::panicking::panic
             at /rustc/8ede3aae28fe6e4d52b38157d7bfe0d3bceef225/library/core/src/panicking.rs:117:5
   3: ctx::update_factors
   4: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   5: run_user_code_8
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
